# Radiospecta

* Sunpy provisionally affiliated package.
* Provides `Fido` clients for a number of space and ground based observatories or observatory networks
  * RSTN, EOVSA, WIND/WAVES, STEREO WAVES, Parker Solar Probe/RFS, CALLISTO
* Provides a `Spectrogram` object or container like `TimeSeries` or `Map` but for dynamic spectra or spectrograms
* Docs https://docs.sunpy.org/projects/radiospectra/en/latest/
* Github https://github.com/sunpy/radiospectra

### Today will cover
1. Search and download data `Fido`
2. Loading data into spectrogram container `Spectrogram`
3. Plotting individual spectra
4. Loading custom data into a Spectrogram
5. Make a combined plot from 1GhZ to 1kHz

## Imports

In [ ]:
from datetime import datetime

import astropy.units as u
import numpy as np

from astropy.time import Time
from astropy.io import fits
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm

from sunpy.net import Fido, attrs as a

from radiospectra import net #let Fido know about the radio clients
from radiospectra.spectrogram2 import Spectrogram # in the process of updating old spectrogram

# What can be searched for?

In [ ]:
Fido

In [ ]:
a.Instrument.waves   #try .waves .swave .rstn .ecallisto

# Lets look for a lot of radio data from 2017-09-02 15:00 - 18:00
* Use `Fido` like yesterday but specifiy different instruments
* `|` can be used as `or` so instrument `a or b` is `a | b`
* Search for data from WIND/WAVES, RSTN, CALLISTO, and SWAVES
* Small bug the name of the client and attr for Callisto don't match `CALLISTO` verses `ecallisto`

In [ ]:
query = Fido.search(a.Time('2017-09-02T15:00', '2017-09-02T18:00'), 
                    a.Instrument.waves | a.Instrument.rstn | a.Instrument.ecallisto | a.Instrument.swaves)

Print the results and see what was found

In [ ]:
query

# Many results let try RSTN

* First download the the data
* Same as yesterday can just pass the query to `Fido.fetch`

In [ ]:
rstn_files = Fido.fetch(query['rstn'])

# using index
# rstn_files = Fido.fetch(query[1])

The `Fido.fetch` call basially returns a lis of filepaths e.g

In [ ]:
rstn_files

## Load into spectrograms 

* Very similar to `Map` and `TimeSeries` the `Spectrogram` can take a list of:
   * data header pairs
   * filepaths
   * urls
   * or a combinatino of the above
* Since fido returns more or less a list of file paths we can just pass this directly to `Spectrogram`

In [ ]:
rstn_spec = Spectrogram(sorted(rstn_files))

Since we gave it a list of filepaths should get back a list of spectrograms

In [ ]:
type(rstn_spec), len(rstn_spec)

Finnally spectrogram hava short repr or text representation

In [ ]:
rstn_spec[0]

Once the data is loaded into a spectrogram it can be easily plotted using the `plot` method and the plots
can be adjusted like a normal matplotlib plt 

In [ ]:
rstn_spec[0].plot()

In [ ]:
rstn_spec[1].plot()

There isn't an obvious radio busrt in the RSTN data so move on to WAVES

## Now do the same steps but for waves
* Remeber you can access the waves part of the query using `query['waves']`

Download WAVES

In [ ]:
# Code to download the waves do here

waves_files = Fido.fetch(query['waves'])

Load into spectrogram

In [ ]:
# Load
waves_spec = Spectrogram(sorted(waves_files))

What is the type and size of the result

In [ ]:
# Check the type and length here
type(waves_spec), len(waves_spec)

Print a string reprentation a WAVES spectrogmram

In [ ]:
waves_spec[0]

Plot the WAVES spectrogram

In [ ]:
waves_spec[0].plot()

In [ ]:
waves_spec[1].plot()

### Waves data looks like it captured a burst lets make a nicer figure

In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True)

waves_spec[0].plot(axes=axes[0])
waves_spec[1].plot(axes=axes[1])

In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True)

waves_spec[0].plot(axes=axes[0])
waves_spec[1].plot(axes=axes[1])

# by default y-axis low to hight flip so moving away fro sun with time
axes[0].set_ylim(reversed(axes[0].get_ylim()))
axes[1].set_ylim(reversed(axes[1].get_ylim()))


In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True)

waves_spec[0].plot(axes=axes[0])
waves_spec[1].plot(axes=axes[1])

# by default y-axis low to hight flip so moving away fro sun with time
axes[0].set_ylim(reversed(axes[0].get_ylim()))
axes[1].set_ylim(reversed(axes[1].get_ylim()))

# log y-axis
axes[0].set_yscale('log')
axes[1].set_yscale('log')

In [ ]:
from datetime import datetime

fig, axes = plt.subplots(2, 1, sharex=True)

waves_spec[0].plot(axes=axes[0])
waves_spec[1].plot(axes=axes[1])

# by default y-axis low to hight flip so moving away fro sun with time
axes[0].set_ylim(reversed(axes[0].get_ylim()))
axes[1].set_ylim(reversed(axes[1].get_ylim()))

# log y-axis
axes[0].set_yscale('log')
axes[1].set_yscale('log')

axes[1].set_xlim(datetime(2017, 9, 2, 15), datetime(2017, 9, 2, 18))

# Find some callisto data?
* http://soleil.i4ds.ch/solarradio/callistoQuicklooks/?date=20170902
* Glasgow data looks good, what about others, Greenland?

In [ ]:
index = ((query['CALLISTO']['Observatory'] == 'GREENLAND') 
         & (query['CALLISTO']['ID'] == 62))

# Glasgow is also intersting
# index = ((query['CALLISTO']['Observatory'] == 'GLASGOW') 

query['CALLISTO'][index]

In [ ]:
callisto_files = Fido.fetch(query['CALLISTO'][index], max_conn=1)
callisto_files = Fido.fetch(callisto_files)

In [ ]:
callisto_specs = Spectrogram(sorted(callisto_files), silence_errors=True)

callisto_specs

In [ ]:
fig, axes = plt.subplots()
[sp.plot(axes=axes) for sp in callisto_specs]
# order not guarnteed so need to set plot range 
axes.set_xlim(datetime(2017, 9, 2, 15, 25), datetime(2017, 9, 2, 18, 10))

# Load data not supported by Spectrogram

In [ ]:
data = np.zeros((200, 200))
data[50:150, 50:150] = 1
times = Time('2017-09-02 15:00') + np.arange(200) * u.s
freqs = np.linspace(10, 20, 200) * u.GHz

# expects at least the following meta data
meta = {
    'observatory': 'some place',
    'instrument': 'an instrument',
    'detector': 'the detector',
    'freqs': freqs,
    'times': times,
    'wavelength': a.Wavelength(freqs[0], freqs[-1]),
    'start_time': times[0],
    'end_time': times[-1]
}
super_spec = Spectrogram(data, meta )

In [ ]:
super_spec

In [ ]:
super_spec.plot()

# Use the Orfees web site to find and dowload data
* Download data to you local machine
* https://rsdb.obs-nancay.fr/

In [ ]:
orfees = fits.open('https://drive.google.com/uc?export=download&id=1vcRra0MAUzEbjphwXK8hBFiP8QM7wNOV')
# orfees = fits.open(<path goes here>)
orfees

In [ ]:
orfees_i = np.hstack([orfees[2].data[f'STOKESI_B{i}'] for i in range(1, 6)]).T
orfees_v = np.hstack([orfees[2].data[f'STOKESV_B{i}'] for i in range(1, 6)]).T

orfees_time_str = orfees[0].header['DATE-OBS']

orfees_times = Time(orfees_time_str) + (orfees[2].data['TIME_B1']/1000)*u.s # times are not the same for all sub spectra!
orfees_freqs = np.hstack([orfees[1].data[f'FREQ_B{i}'] for i in range(1, 6)]) *u.MHz

In [ ]:
orfees_meta = {
    'observatory': orfees[0].header['ORIGIN'],
    'instrument': orfees[0].header['INSTRUME'],
    'detector': orfees[0].header['INSTRUME'],
    'freqs': orfees_freqs.reshape(-1),
    'times': orfees_times,
    'wavelength': a.Wavelength(orfees_freqs[0,0], orfees_freqs[0,-1]),
    'start_time': orfees_times[0],
    'end_time': orfees_times[-1]
}
orfees_spec_i = Spectrogram(orfees_i, orfees_meta)
orfees_spec_v = Spectrogram(orfees_v, orfees_meta)
orfees_spec_i

In [ ]:
vmm = np.percentile(orfees_spec_i.data, [1,99])
orfees_spec_i.plot(norm=LogNorm(vmin=vmm[0], vmax=vmm[1]))


In [ ]:
vmm = np.percentile(orfees_spec_v.data, [1,99])
orfees_spec_v.plot(vmin=vmm[0], vmax=vmm[1])
plt.semilogy()
plt.ylim(plt.ylim()[::-1])

# Final combined plot 1 GHz to 1 kHz

In [ ]:
from datetime import datetime
from matplotlib.colors import LogNorm

# set up the plots 4 rows 1 column with shared common x-axis
fig, axes = plt.subplots(4, 1, sharex=True, figsize=(12, 12), dpi=100)

waves_spec[0].plot(axes=axes[0], norm=LogNorm())
waves_spec[1].plot(axes=axes[1], norm=LogNorm())

[sp.plot(axes=axes[2]) for sp in callisto_specs]

vmm = np.percentile(orfees_spec_i.data, [1,99])
orfees_spec_i.plot(axes=axes[3], vmin=vmm[0], vmax=vmm[1])

# by default y-axis low to high flip so moving away from sun with time
[axes[i].set_ylim(reversed(axes[i].get_ylim())) for i in range(4)]

# log y-axis
[axes[i].set_yscale('log') for i in range(4)]

# reduce time range
axes[1].set_xlim(datetime(2017, 9, 2, 15, 0), datetime(2017, 9, 2, 17, 30))